# DiscRimNN 
### By Willie Maddox
## Problem A
Create a model that reads a sequence of numbers and classifies the last number in the sequence based on the previous numbers in the sequence.

## Problem B
Create a model that reads a sequence of numbers, one number at a time, and classify that number based on all previously seen numbers.

## Build a signal generator

The signal generator builds waves using the standard form:

$$x(t) = h + A\sin\left(\frac{2\pi t}{T} + \phi\right)$$

where $h$ is the height (vertical offset), $A$ is the amplitude (vertical scale), $T$ is the period (horizontal scale), and $\phi$ is the phase (horizontal offset). An optional $\Delta$ value can also be assigned to each of these 4 variables. These variables give us fine grained control over how we construct our waves and the $\Delta$ values allow us to introduce randomness in the variables.

In [ ]:
from discrimnn.signal import MixedSignal

# Trivial Case: Offset (-3, 0, 3)

<img src="out/offset_30/signals.png" width="1000">

<img src="out/offset_30/mixedsignal.png" width="1000">

<img src="out/offset_30/mixedsignal_with_truth.png" width="1000">

In [ ]:
# start off with simplest case for proof of concept
time_coeffs = {'start': 0, 'stop': 75, 'n_timestamps': 301, 'n_timesteps': 10}

sig1_coeffs = {'name': 'A', 'offset': {'mean': -3}, 'color': '#ff0000'}
sig2_coeffs = {'name': 'B', 'offset': {'mean': 0}, 'color': '#00ff00'}
sig3_coeffs = {'name': 'C', 'offset': {'mean': 3}, 'color': '#0000ff'}
sig_coeffs = [sig1_coeffs, sig2_coeffs, sig3_coeffs]

msig_coeffs = {'phase': {'mean': 0, 'delta': np.pi}, 'period': {'mean': 25}}

msig = MixedSignal(time_coeffs, sig_coeffs, msig_coeffs, method='sliding')

<img src="out/offset_30/mixedsignal_table.png" width="300">

## Next define the RNN

```python
model = Sequential()
model.add(LSTM(32, batch_input_shape=(4, 10, 1), stateful=True, return_sequences=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(n_signals, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()
```

<img src="out/offset_30/model_plot.png" width="400">

## Now train the model

```python
n_generations = 20
for i in range(n_generations):
    X, y = msig.generate()
    history = model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
    model.reset_states()
```

## Results (training loss, accuracy, and test)

<img src="out/offset_30/loss_acc.png" width="1000">

<img src="out/offset_30/eval_pred.png" width="1000">

# Offset (-0.5, 0, 0.5)

<img src="out/offset_050/signals.png" width="1000">
<img src="out/offset_050/mixedsignal.png" width="1000">
<img src="out/offset_050/mixedsignal_with_truth.png" width="1000">

# Offset (-0.5, 0, 0.5) - Results

<img src="out/offset_050/loss_acc.png" width="1000">
<img src="out/offset_050/eval_pred.png" width="1000">

# Offset (-0.1, 0, 0.1)

<img src="out/offset_010/signals.png" width="1000">

<img src="out/offset_010/mixedsignal.png" width="1000">

<img src="out/offset_010/mixedsignal_with_truth.png" width="1000">

# Offset (-0.1, 0, 0.1) - Results

<img src="out/offset_010/loss_acc.png" width="1000">
<img src="out/offset_010/eval_pred.png" width="1000">

# Phase (0, $\pi$, 0)

<img src="out/phase_010/signals.png" width="1000">
<img src="out/phase_010/mixedsignal.png" width="1000">
<img src="out/phase_010/mixedsignal_with_truth.png" width="1000">

# Phase (0, $\pi$, 0) - Results

<img src="out/phase_010/loss_acc.png" width="1000">
<img src="out/phase_010/eval_pred.png" width="1000">

# Summary
 1. Full control over dataset
 2. Dataset is infinite
 3. Model never sees the same sample twice.
 4. Can study all three RNN base problems (classification, prediction, forcasting).
 5. Great way to zero in on and study a particular aspect of RNN's

# Future work (TODO list)
 - [x] create single signal generator class
 - [x] create mixed signal generator class
 - [ ] create signal noise functions (Gaussian, OU, etc.)
 - [ ] create timestep noise functions 
 - [x] add legends to plots.
 - [ ] during training, save outlier X, y train sets to file for further analysis.
 - [x] save configuration of mixed signal properties as json for housekeeping.
 - [ ] make plots of the mixed signal with colors mapped to hidden layers, lstm states, etc.
 - [ ] unit tests for signal.py
 - [ ] create startup.py to handle project directories and other goodies.
 - [ ] fix savefig clipping the bottoms of our figures.